# Exercise 5: Blocks and inlined rules

This notebook introduces more complex language structures that are able to restrict the matching context or even extend the expressivity of the matching process. 

#### Setup

In [ ]:
%%documentText
Peter works for Frank.
10€ are less than 100$.

In [ ]:
DECLARE Sentence;
(ANY+{-PARTOF(Sentence),-PARTOF(PERIOD)} PERIOD){-> Sentence};

### The `BLOCK` construct

Suppose, our goal is to annotate all pairs of tokens within sentences (not across sentences) for sentences that contain a number. This can be solved using a `BLOCK` construct. The `BLOCK` construct iterates over all annotations of a given type (`Sentence` in this case). The condition `{CONTAINS(NUM)}` limits the sentences to those that contain a number. It can also be left empty `{}` if one wants to iterate over all `Sentence` annotations. The name of the `BLOCK` is `sentence` (but not further used in this example).

All rules within the `BLOCK` construct are then limited to the Tokens of the given match, i.e. in our case, they are limited to the current sentence. Please observe that the TokenPair1 annotations are only pairs that exist within the second sentence.

In [ ]:
%displayMode CSV
%csvConfig TokenPair1
DECLARE TokenPair1;

BLOCK(sentence) Sentence{CONTAINS(NUM)}{
    (ANY ANY){-> TokenPair1};
}

`BLOCK` can also be used as an if statement (see below), as a loop (including recursion), as named procedures and more.

### Inlined rule as action

We do the same using an inlined rule as action. Inlined rules as action indicated by `->` after the rule element can apply additional rules within the scope of the match of a rule element, if the complete rule matched successfully.

In [ ]:
%csvConfig TokenPair2

DECLARE TokenPair2;

Sentence->{
    (ANY ANY){-> TokenPair2};
};

### Using `BLOCK` like an if statement.

Sometimes, it is useful to limit the analyses to certain documents. The following code block illustrates how only documents that contain a number — please note that this could of course be any kind of condition such as a specific identifier or the presence/absensce of a certain annotation — will be analyzed and a set of rules (here: extracting word tuples for each sentence) are applied.

In [ ]:
%csvConfig TokenPair3

DECLARE TokenPair3;

BLOCK(doc) Document{CONTAINS(NUM)}{
    Sentence->{
        (ANY ANY){-> TokenPair3};
    };
}

### Inlined rule as condition

Next, we annotate all sentences that contain a `WorksFor` relation with an employee named "Peter". 

Complex conditions like these can be specified using inlined rules as condition. Rule elements extended with these inlined rules indicated with `<-` only match if at least one rule within the inlined block is able to match. In this scenario, we add a `SentenceWithRelation` annotation to each `Sentence` if it contains a `WorksFor` relation with an employee named "Peter". 

In [ ]:
%csvConfig SentenceWithRelation
DECLARE Employer, Employee;
"Peter"-> Employee;
"Frank"-> Employer;

DECLARE WorksFor (Employee employee, Employer employer);
(e1:Employee # e2:Employer){-> wf:WorksFor, wf.employee=e1, wf.employer=e2};

DECLARE SentenceWithRelation;

Sentence{-> SentenceWithRelation}
    <-{WorksFor.employee.ct=="Peter";};


Now, we annotate all sentences that are followed by a sentence that contain a `MoneyAmount` in Euro. These inlined rules can of course also be used in sequential patterns and they can also be used within other inlined rule blocks. Multiple inlined rule blocks at one rule element are also supported. If multiple inlined rule as condition block are given at one rule element at least one rule in each of them needs to be able to match, i.e. they specify a conjunction.

In [ ]:
%csvConfig SentenceFollowedByMoneySentence
DECLARE MoneyAmount(INT amount, STRING currency);
INT value;
(NUM{PARSE(value)} c:SPECIAL){-> CREATE(MoneyAmount, "amount"=value, "currency"=c.ct)};

DECLARE SentenceFollowedByMoneySentence;

Sentence{-> SentenceFollowedByMoneySentence} Sentence<-{MoneyAmount.currency.ct=="€";};


### The `FOREACH` block

We annotate each word that is either followed or preceded by a capital-written word using a `FOREACH` block. The `FOREACH` block is a special block, which appears similar to the `BLOCK` presented before, but has some subtle differences. Similar to before, it assigns the annotation matched by the head rule to a temporary variables (`w` in our case). This annotation variable can be used with the block. 

The main difference to the `BLOCK` is that the `FOREACH` block does not restrict the window for the contained rules. More specifically, in our example the scope of the words (`w`) is not limited to the word itself (this would be the case in a `BLOCK` construct), but rules can also access preceding and following annotations.

The `FOREACH` block is very similar to a loop over a defined set of annotations. It iterates from left to right, but can also iterate from right to left given an additional argument. This block construct can be used for more efficient rule executions (like a simple FST automata) since the variable can be matched directly without iterating over the index anew.

In [ ]:
%displayMode RUTA_COLORING

DECLARE Word;

FOREACH(w) W{}{
    w{->Word} CW;
    CW w{-> Word};
}

COLOR(Word, "pink");
